Import des bibliothèques

In [1]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc
import plotly.express as px

#3 methodes de projection et de clustering avec justifications
# tableau de comparaison des méthodes de clustering avec si oui ou non on detecte une classe donnée avec toutes les méthodes

Ouverture du CSV

In [ ]:
data=pd.read_csv("binding_site_database_smallest.csv")
data.describe()

,model,carbohydrate_type,polar_sasa,apolar_sasa,total_sasa,residues_in_helices,residues_in_strands,residues_in_turns,residues_in_bends,residues_in_loops,...,count_M,count_N,count_P,count_Q,count_R,count_S,count_T,count_V,count_W,count_Y
count,3953.000000,3953.000000,3953.000000,3953.000000,3953.000000,3953.000000,3953.0,3953.000000,3953.000000,3953.000000,...,3953.000000,3953.000000,3953.000000,3953.000000,3953.000000,3953.000000,3953.000000,3953.000000,3953.000000,3953.000000
mean,1.001265,1.492537,141.938490,156.074345,298.012835,2.158108,0.0,1.185935,1.359221,2.139641,...,0.144953,0.841892,0.260056,0.357703,0.687832,0.542879,0.554769,0.344801,0.477106,0.673413
std,0.047705,0.755216,62.937028,76.478857,111.005253,2.878801,0.0,1.374686,1.634947,1.986093,...,0.413567,0.887124,0.547044,0.632737,0.986085,0.822412,0.806162,0.598377,0.828420,0.890994
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,99.319339,103.205044,229.563881,0.000000,0.0,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,141.129539,154.360836,306.384007,1.000000,0.0,1.000000,1.000000,2.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,2.000000,182.948750,205.038135,371.599984,3.000000,0.0,2.000000,2.000000,3.000000,...,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,3.000000,3.000000,404.728287,483.885263,679.644289,20.000000,0.0,12.000000,12.000000,24.000000,...,4.000000,7.000000,4.000000,5.000000,6.000000,5.000000,5.000000,4.000000,6.000000,7.000000


In [ ]:
data.head()

,pdb_id,model,carbohydrate_name,carbohydrate_label,carbohydrate_type,polar_sasa,apolar_sasa,total_sasa,residues_in_helices,residues_in_strands,...,count_N,count_P,count_Q,count_R,count_S,count_T,count_V,count_W,count_Y,function
0,6STO,1,GAL,GAL:2:C,1,194.853224,171.415915,366.269139,1,0,...,0,0,0,0,0,0,0,0,2,Sugar binding protein
1,1YF8,1,GAL,GAL:1013:B,1,226.262482,271.041979,497.304461,2,0,...,1,1,1,1,0,0,0,1,0,Toxin
2,5AJC,1,GAL,GAL:2:F,1,44.752831,134.664164,179.416996,0,0,...,0,0,0,0,0,1,0,1,1,Antibiotic
3,4UP4,1,GAL,GAL:1:C,1,116.656391,187.623798,304.280189,0,0,...,1,0,0,0,0,0,0,1,0,Lectin
4,2IWG,1,GAL,GAL:6:C,3,190.064219,175.539590,365.603809,5,0,...,0,3,0,0,0,2,1,0,0,Immune system


Création d'un sous jeu de données pour études préliminaires

In [ ]:
selected_column=["carbohydrate_name","carbohydrate_type","polar_sasa","apolar_sasa","total_sasa","residues_in_helices","residues_in_strands","residues_in_turns","residues_in_bends","residues_in_loops"]
df=data[selected_column]

Etude préliminaire: Boxplot

In [ ]:
fig = px.box(df, x=selected_column[1:])
fig.show()

Nous allons normaliser les variables car il y a de grandes différences de distributions

In [ ]:
scaler = StandardScaler()  

df = pd.DataFrame(scaler.fit_transform(df.iloc[:,1:]),columns=selected_column[1:])


In [ ]:
df.head()

,carbohydrate_type,polar_sasa,apolar_sasa,total_sasa,residues_in_helices,residues_in_strands,residues_in_turns,residues_in_bends,residues_in_loops
0,-0.652263,0.840863,0.200624,0.614970,-0.402339,0.0,-0.862804,1.003694,-0.573883
1,-0.652263,1.339985,1.503451,1.795562,-0.054928,0.0,-0.135273,-0.219742,0.433247
2,-0.652263,-1.544368,-0.279984,-1.068516,-0.749750,0.0,-0.862804,-0.831460,-0.573883
3,-0.652263,-0.401756,0.412577,0.056467,-0.749750,0.0,-0.862804,1.003694,0.433247
4,1.996320,0.764762,0.254550,0.608976,0.987304,0.0,-0.862804,-0.831460,-0.070318


In [ ]:


fig = px.box(df, x=selected_column[1:])
fig.show()

Histogramme Nombre de résidus par type de résidus

In [ ]:
fig = px.histogram(data, x="carbohydrate_name").update_xaxes(categoryorder='total descending')
fig.show()

Etude préliminaire: Dendogramme

In [ ]:
plt.figure(figsize=(10, 7))
plt.title("Clustering of properties in binding sites")

selected_data = df
clusters = shc.linkage(selected_data, 
            method='ward', 
            metric="euclidean")
shc.dendrogram(Z=clusters)
plt.show()

2 Clusters